### Load libraries and functions

In [4]:
#Load libraries
library(slingshot)
library(ggsci)
library(scales)
library(tidyr)
library(Seurat)
library(Rcpp)
library(parallelDist)
library(stringr)
library(viridis)
library(ggplot2)
library(cowplot)
library(cluster)
library(data.table)
library(foreach)
library(doParallel)
library(proxy)
library(ComplexHeatmap)
library(circlize)
library(igraph)
library(qvalue)
library(dplyr)
library(viridis)
library(VGAM)
library(forcats)
library(grDevices)
library(graphics)
library(RColorBrewer)
library(pheatmap)
library(Cairo)
library(reshape2)
library(R.utils)
set.seed(seed = 42)

In [5]:
#load basic functions
createEmptyDf = function( nrow, ncol, colnames = c() ){
  data.frame( matrix( vector(), nrow, ncol, dimnames = list( c(), colnames ) ) )
}

tableread_fast = function(i, header=TRUE, quote="", sep=","){
  tmp = fread(i, header=header, sep=sep, quote=quote, nThread=32)
  tmp = as.data.frame(tmp)
  return(tmp)
}

### Create Seurat object

In [6]:
#load data: aCD4 SCT
#Put gene list in the target panel in the working directory ("BD_genes_plus10")

###Input layer
set.seed(seed = 42)
input_dir <- "data/SCT_rawdata"
sample.name <- "aCD4"

################################ Processing layer ##########################################
#Create input file name
input_name <- str_c(input_dir, "matrix_inflection_demulti_", sep = "/") %>% str_c(sample.name, ".txt", sep = "") #Name of dataset1

#Unzip input file
compressed.name <- str_c(input_name, ".gz", sep = "")
gunzip(compressed.name)

#Create Seurat object
matrix <- tableread_fast(input_name, header = TRUE, quote="", sep="\t")
row.names(matrix) <- matrix$V1
matrix <- dplyr::select(matrix, -V1)
seu1 <- CreateSeuratObject(counts=matrix, project = "seu", min.cells = 3, min.features = 10)
gzip(str_remove(compressed.name, ".gz"))
unlink(str_remove(compressed.name, ".gz"))

In [7]:
#load data: Control/aPDL1 SCT
#Put gene list in the target panel in the working directory ("BD_genes_plus10")

###Input layer
set.seed(seed = 42)

sample.name <- "Cont_aPDL1"

################################ Processing layer ##########################################
#Create input file name
input_name <- str_c(input_dir, "matrix_inflection_demulti_", sep = "/") %>% str_c(sample.name, ".txt", sep = "") #Name of dataset1

#Unzip input file
compressed.name <- str_c(input_name, ".gz", sep = "")
gunzip(compressed.name)

#Create Seurat object
matrix <- tableread_fast(input_name, header = TRUE, quote="", sep="\t")
row.names(matrix) <- matrix$V1
matrix <- dplyr::select(matrix, -V1)
seu2 <- CreateSeuratObject(counts=matrix, project = "seu", min.cells = 3, min.features = 10)
gzip(str_remove(compressed.name, ".gz"))
unlink(str_remove(compressed.name, ".gz"))

### Preprocessing for excluding check non-T cell contamination

In [8]:
###Function: perform Seurat pipeline: Normalizing, Scaling, PCA, JackStraw, dimentional reduction, find clusters
##Input: seurat object
##Output: seurat object, JackStraw Plot

Preprocess <- function(seu, dir.name, sample.name){
    # normalizing data
    seu = NormalizeData(object = seu, scale.factor=1000000)
    VariableFeatures(seu) <- rownames(seu[["RNA"]]) #Variable features: all genes
    ngenes <- length(x = seu@assays$RNA@var.features)
    all.genes <- rownames(seu)
    seu = ScaleData(object = seu, vars.to.regress = c("nCount_RNA"), features = all.genes) 

    # perform PCA
    seu = RunPCA(object = seu, features = seu@assays$RNA@var.features, npcs = 50)

    #JackStraw
    seu = JackStraw(object = seu, num.replicate = 100, dims = 50)
    seu <- ScoreJackStraw(object = seu, dims = 1:50, score.thresh = 0.05)
    file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("_JackStraw.png", sep='')
    png(file.name, width = 1250, height = 500)
    p <- JackStrawPlot(object = seu, dims = 1:50)
    plot(p)
    dev.off()

    #Determine PCs used for clustering/tSNE analysis (dims.use)
    #Extract PCs which fulfill the pvalue threshold
    tmp = as.data.frame(seu@reductions$pca@jackstraw@overall.p.values)
    tmp1 = tmp[tmp$Score>0.05,1]
    dims= c(1:(min(tmp1)-1))

    ##Perform dimentional reduction
    seu <- RunUMAP(object = seu, dims = 1:(min(tmp1)-1))

    ##Find clusters
    seu = FindNeighbors(seu, reduction = "pca", dims = dims, force.recalc = TRUE)
    
    return(seu)
}


In [9]:
###Function: DimPlot with sample origin
##Input: seurat object
##Output: DimplotStraw Plot

DimOrigin <- function(seu, dir.name, sample.name, red.use, resol){
    p1 = DimPlot(object = seu, reduction = red.use, label = TRUE, label.size = 10, pt.size = 0.5) +
      theme(axis.title.x = element_text(size=10, family = "Arial"), 
            axis.title.y = element_text(size=10, family = "Arial"), 
            axis.text.x = element_text(size=10, colour = 1, family = "Arial"), 
            axis.text.y = element_text(size = 10, colour = 1, family = "Arial")) +
      theme(panel.border = element_rect(fill = NA, size = 1)) 
    p2 = DimPlot(object = seu, reduction = red.use, label = FALSE, label.size = 10, pt.size = 0.5, group.by = "orig.ident") +
      theme(axis.title.x = element_text(size=10, family = "Arial"),
            axis.title.y = element_text(size=10, family = "Arial"),
            axis.text.x = element_text(size=10, colour = 1, family = "Arial"),
            axis.text.y = element_text(size = 10, colour = 1, family = "Arial")) +
      theme(panel.border = element_rect(fill = NA, size = 1)) 

    legend1 <- cowplot::get_legend(p1)
    legend2 <- cowplot::get_legend(p2)
    p1 = p1 + theme(legend.position = 'none')
    p2 = p2 + theme(legend.position = 'none')
    file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c(red.use, "reso", resol, "png", sep='.')
    save_plot(file = file.name, plot_grid(p1, legend1, p2, legend2, ncol=2, nrow=2), device="png", 
              units="in", dpi = 600, base_width = 10, base_height = 10, limitsize=FALSE)
}

In [10]:
###Function: Find marker genes with marker heatmap
##Input: seurat object
##Output: Marker gene table and marker gene heatmap plot

MarkerHeatmap <- function(seu, dir.name, sample.name, resol){
    seu.markers = FindAllMarkers(seu, verbose = TRUE, test.use="wilcox", only.pos=TRUE, min.pct=0.1, features.use = NULL, return.thresh=0.05)

    #Create heatmap with top10 marker genes
    top10 = seu.markers %>% group_by(cluster) %>% top_n(10, avg_log2FC)
    top10 = as.data.frame(top10)
    top10  = top10 [!duplicated(top10$gene),]
    top10 = top10 %>% arrange(desc(avg_log2FC))  %>% arrange(cluster)
    top10 = as.data.frame(top10)
    file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("marker_res", resol, "png", sep='.')
    p <- DoHeatmap(seu, features = top10$gene, disp.min = -2.5, disp.max = 2.5, size = 8)
    ggsave(file = file.name, plot = p, device="png", units="in", dpi = 300,
           width = 20, height = 20, limitsize=FALSE)

    seu.markers$cluster = as.numeric(seu.markers$cluster)
    seu.markers = seu.markers %>% arrange(desc(avg_log2FC))  %>% arrange(cluster)
    seu.markers = as.data.frame(seu.markers)
    file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("ALLmarkers_minpct0.1_Adj_p0.05.txt", sep='')
    fwrite(seu.markers, file.name, row.names=F, col.names=T, sep="\t", quote=F)
    file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("ALLmarkers_minpct0.1_Adj_p0.05.rda", sep='')
    save(seu.markers, file=file.name)
}

In [11]:
###Function: Create Scatter and Violin plot for selected genes
##Input: seurat object, gene list
##Output: Scatter plot, Violin plot

ScatterViolin <- function(seu, dir.name, sample.name, red.use, tmps, tmp_names){
    for (i in 1:length(tmp_names)){
        #Call gene list
        tmp <- tmps[[i]]
        tmp_name <- tmp_names[i]
  
        #Scatter Plot
        file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("Scatter", tmp_name, "png", sep='.')
        png(file.name, width = 1536, height = 240)
        p <- FeaturePlot(seu, features = tmp, ncol = 6, order = TRUE,
                        reduction = red.use, dims=c(1,2), cols = c("grey", "red"), pt.size = 0.2)
        plot(p)
        dev.off()
        
        #Violin Plot
        file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("Violin", tmp_name, "png", sep='.')
        png(file.name, width = 1536, height = 240)
        p <- VlnPlot(seu, features = tmp, ncol = 6, pt.size = 0.1)
        plot(p)
        dev.off()
    }
}


### Quality Check

In [12]:
#Merge two datasets
seu <- merge(x=seu1, y = seu2)

In [13]:
###Gene filtering

##Input layer
sample.name <- "Pooled"
dir.name="results/preprocess/Seurat_plot"
gene.list.name <- "BD_genes_plus10.txt"

########################## Processing layer ###############################
dir.name <- str_c(dir.name, sep = "/") %>% str_c("Stat", sep = "/")
dir.create(dir.name, recursive = TRUE)

##load gene list of BD target panel
BD_genes <- read.table(gene.list.name, header = TRUE)
BD_genes <- as.vector(BD_genes$Genesymbol)

##Chose BD target gene list for downstream analysis
res1 = seu@assays$RNA@counts
res1 =res1[res1@Dimnames[[1]] %in% BD_genes,]
ngenes <- length(res1@Dimnames[[1]])
seu@assays$RNA@counts <- res1
seu@assays$RNA@data <- res1
    
#Remove doublet and Pmel cells
seu <- subset(seu, subset = orig.ident %in% c("mouseSampleTag1", "mouseSampleTag2",
                                                "mouseSampleTag5", "mouseSampleTag6"))

#Scatter plot for gene/read count
file.name=str_c(dir.name, sample.name, sep = "/") %>% str_c("Reads.Genes.png", sep='.')
png(file.name, width = 512, height = 400)
p <- FeatureScatter(object = seu, feature1 = "nCount_RNA", feature2 = "nFeature_RNA", pt.size = 0.3) +
      geom_hline(yintercept=10) +
      scale_x_log10()
plot(p)
dev.off()

#RidgePlot for gene/read count, 
file.name=str_c(dir.name, sample.name, sep = "/") %>% str_c("Reads.png", sep='.')
png(file.name, width = 512, height = 400)
p <- RidgePlot(object = seu, features = "nCount_RNA", group.by="orig.ident", ncol = 1) +
      scale_x_log10()
plot(p)
dev.off()

file.name=str_c(dir.name, sample.name, sep = "/") %>% str_c("Genes.png", sep='.')
png(file.name, width = 512, height = 400)
p <- RidgePlot(object = seu, features = "nFeature_RNA", group.by="orig.ident", ncol = 1)
plot(p)
dev.off()

png 
  2

Scale for 'x' is already present. Adding another scale for 'x', which will
replace the existing scale.

Picking joint bandwidth of 0.0332



png 
  2

Picking joint bandwidth of 2.91



png 
  2

###  Initial analysis

In [14]:
###Run Seurat pipeline (dimentional reduction ~ determining resolution) with iterative process

###Input layer 
sample.name <- "Pooled"
dir.name <- "results/preprocess/Seurat_plot"
dir.seurat <- "results/preprocess"
red.use <- "umap"
resol <- 1.0

#For lineage check
tmp_1 = c("Trbc2", "Cd3e", "Cd8a", "Cd4", "Cd14", "Lyz2")
tmps <- list(tmp_1)
tmp_names <- c("lineage")

########################## Processing layer #############################
### Start processing ###
seu <- Preprocess(seu, dir.name, sample.name)
    
#Find clusters
seu <- FindClusters(object = seu, resolution =resol)
    
#DimPlot with sample origin
DimOrigin(seu, dir.name, sample.name, red.use, resol)
    
#Marker gene extraction and create marker gene heatmap
MarkerHeatmap(seu, dir.name, sample.name, resol)
    
#Check contamination
ScatterViolin(seu, dir.name, sample.name, red.use, tmps, tmp_names)
        
#Output Seurat object in 1st analysis
file.name <- str_c(dir.seurat, sample.name, sep = "/") %>% str_c(".rda", sep='')
save(seu, file=file.name)

Regressing out nCount_RNA

Centering and scaling data matrix

PC_ 1 
Positive:  Pdcd1, Lag3, Cxcr6, Lgals1, Ctla4, Gzmb, Bcl2a1a, Nkg7, Klrc1, S100a10 
	   Havcr2, Ifng, Ctsd, Lgals3, Cd8a, Icos, Cd48, Ccl5, Ccr2, Cst7 
	   Gzmk, Ybx3, Itgb2, Ccr5, Cd3e, Klrk1, Tigit, Tnfrsf9, Cd5, Cd3g 
Negative:  Tcf7, Il7r, Ccr7, Sell, Btg1, Lef1, Il4ra, Il6ra, H2-K1, Txk 
	   Arl4c, Cd14, Adgre1, Fcgr3, Cd300a, Pik3ip1, Cxcl2, Cd63, Trem2, Il1rn 
	   Nt5e, Tnfsf13, Itk, Ly86, Cd36, Slc11a1, Clec4d, Mgst1, Ccl2, Cxcl16 
PC_ 2 
Positive:  Lyz2, Cd14, H2-Aa, Fcgr3, Ccl6, H2-Eb1, Ly86, Il1rn, Ifi204, Cxcl2 
	   Adgre1, H2-Ab1, C1qb, Slc11a1, Itgam, Ccr1, Ccl2, Cd63, Cxcl16, Trem2 
	   Ifitm3, Lgals3, Cd300a, Ccl9, C1qa, Nlrp3, Clec4d, Tnfsf13, Cd36, Thbs1 
Negative:  Ccr7, Tcf7, Itk, H2-K1, Il7r, Stat3, Xcl1, Stat4, Trbc2, Sell 
	   Il4ra, Nfkb1, Ptprc, Tnfrsf4, Stat5a, Lef1, Txk, Tnfsf8, Cd3d, Kit 
	   Cxcr5, Nt5e, Cblb, Tfrc, Ighm, Btg1, Nfkb2, Mcm4, Tgfb1, Tnfrsf18 
PC_ 3 
Positive:  Tnfrsf4, Cd74, 

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 35544
Number of edges: 956469

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8151
Number of communities: 17
Elapsed time: 12 seconds


1 singletons identified. 16 final clusters.

Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' not found in PostScript font database”
Warning message in grid.

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warnin

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warnin

### 2nd analysis with removing contaminant non-T cells

In [15]:
#Remove contaminant clusters
seu <- subset(seu, idents = c(0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 13)) #7: NK cells, 12,15:Macrophage, 14:Treg?


In [16]:
###Run Seurat pipeline (dimentional reduction ~ determining resolution) with iterative process

###Input layer 
sample.name <- "Pooled"
dir.name <- "results/preprocess/Seurat_plot_2nd"
red.use <- "umap"
resol <- 1.0

#For lineage check
tmp_1 = c("Trbc2", "Cd3e", "Cd8a", "Cd4", "Cd14", "Lyz2")
tmps <- list(tmp_1)
tmp_names <- c("lineage")

########################## Processing layer #############################
### Start processing ###
dir.create(dir.name)
seu <- Preprocess(seu, dir.name, sample.name)
    
#Find clusters
seu <- FindClusters(object = seu, resolution =resol)
    
#DimPlot with sample origin
DimOrigin(seu, dir.name, sample.name, red.use, resol)
    
#Marker gene extraction and create marker gene heatmap
MarkerHeatmap(seu, dir.name, sample.name, resol)
    
#Check contamination
ScatterViolin(seu, dir.name, sample.name, red.use, tmps, tmp_names)
        
#Output Seurat object in 2nd analysis
file.name <- str_c(dir.seurat, sample.name, sep = "/") %>% str_c("_2nd.rda", sep='')
save(seu, file=file.name)

Regressing out nCount_RNA

Centering and scaling data matrix

Warning message in PrepDR(object = object, features = features, verbose = verbose):
“The following 1 features requested have zero variance (running reduction without them): Arg1”
PC_ 1 
Positive:  Ccr7, Tcf7, Il7r, Sell, Btg1, Lef1, Il4ra, H2-K1, Itk, Txk 
	   Il6ra, Arl4c, Nt5e, Pik3ip1, Stat3, Trib2, Bcl2, Cxcr5, Bach2, Itgae 
	   Stat5a, Myc, Fas, Birc3, Ikbkb, Tnfsf8, H2-Ob, Socs1, Ighm, Stat4 
Negative:  Pdcd1, Gzmb, Cxcr6, Lgals1, Lag3, Ctla4, Lgals3, Nkg7, Ccl5, Bcl2a1a 
	   Ccr2, Ctsd, Klrc1, S100a10, Havcr2, Ifng, Gzmk, Ccr5, Icos, Cd8a 
	   Cd48, Itgb2, Fasl, Klrk1, Cst7, Gimap7, Tigit, Ccl4, Ybx3, Casp1 
PC_ 2 
Positive:  Ccl5, Gzmk, Itgax, Gzma, Gzmb, Ifngr1, Klrg1, Ccr2, Cxcr3, Gimap7 
	   Arl4c, Ifit3b, Btg1, Gzmm, Fasl, Ifit3, Dusp2, Cxcr4, Igkc, Entpd1 
	   Cmklr1, Ccr5, Lgals3, Itga4, Irf7, Lat, Nod1, Rgs1, Cd72, Eomes 
Negative:  Tnfrsf4, Xcl1, Tnfrsf9, Nrp1, Lag3, Irf8, Cd74, Tnfrsf18, Kit, Ccr7 
	   Il1r2

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 32870
Number of edges: 811262

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.7951
Number of communities: 15
Elapsed time: 10 seconds


1 singletons identified. 14 final clusters.

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' 

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warnin

Calculating cluster 11

Calculating cluster 12

Calculating cluster 13



### Determine clustering resolution

In [19]:
###Visualize clusters

###Input layer 
sample.name <- "Pooled"
dir.name <- "results/preprocess/Seurat_plot_Clustering"
red.use <- "umap"
resol <- 0.5

########################## Processing layer #############################
dir.create(dir.name)
    
#Find clusters
seu <- FindClusters(object = seu, resolution =resol)
    
#DimPlot with sample origin
DimOrigin(seu, dir.name, sample.name, red.use, resol)
    
#Marker gene extraction and create marker gene heatmap
MarkerHeatmap(seu, dir.name, sample.name, resol)
    
#Output Seurat object in 2nd analysis
file.name <- str_c(dir.seurat, sample.name, sep = "/") %>% str_c("_clust.rda", sep='')
save(seu, file=file.name)


Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 32870
Number of edges: 811262

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8628
Number of communities: 10
Elapsed time: 10 seconds


1 singletons identified. 9 final clusters.

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' n

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warnin

### Pseudotime Analysis

In [55]:
load("results/preprocess/Pooled_clust.rda")

In [41]:
#Pseudotime analysis
#seu.sub: create pseudotime for subset of clusters
#red.use: dimentional reduction for plotting pseudotime
Pseudotime.Min <- function(seu, seu.sub, name, dims, start, dir.name){
  #Create lineages and pseudotime
  lineages = slingshot(seu.sub@reductions$pca@cell.embeddings[,dims], 
                       clusterLabels = seu.sub@active.ident, maxit=1000, start.clus=start)
  pseudotime=slingPseudotime(lineages, na = TRUE)
  
  #calculate pseudotime for each lineage
  for (i in c(1:length(lineages@metadata$lineages))){
    pseudotime[,i]=pseudotime[,i]/max(pseudotime[,i][!is.na(pseudotime[,i])])
  }
  if (length(lineages@metadata$lineages) == 1) {
    pseudotime1 = pseudotime[,1]
  } else {
    pseudotime1 = Matrix::rowMeans(pseudotime, na.rm=TRUE)
  }
    
    print(SlingshotDataSet(lineages))
  
  #Add to MetaData
  seu2 = AddMetaData(object = seu, metadata = pseudotime1, col.name = "pseudotime")
    
  #Plot pseudotime in dimentional reduction
    name.output=str_c(dir.name, name, sep='/') %>% str_c("from", start, red.use, "pseudotime.tiff", sep = ".")
    ppi <- 600
    tiff(name.output, width=3.8*ppi, height=3*ppi, res=ppi)
    #Output twice size of figure, compress after pasting in the powerpoint.
    p <- FeaturePlot(object = seu2, features = "pseudotime", cols = c("#800080", "#ffee00"), reduction = red.use, pt.size = 0.1) +
          theme(plot.title = element_blank(), 
                axis.title.x = element_text(size=16, family = "Arial"), 
                axis.title.y = element_text(size=16, family = "Arial"), 
                axis.text.x = element_text(size=12, colour = 1, family = "Arial"), 
                axis.text.y = element_text(size = 12, colour = 1, family = "Arial"),
               legend.title = element_text(size=12, family = "Arial"),
               legend.text = element_text(size=12, family = "Arial"),
               legend.key.width = unit(0.5, "cm")) +
          theme(panel.border = element_rect(fill = NA, size = 0.75)) 
    plot(p)
    dev.off()
    
  return(seu2)
}

In [56]:
#Append sample origin to meta.data
seu@meta.data$sample.origin <- NA
seu@meta.data$sample.origin[which(seu@meta.data$orig.ident == "mouseSampleTag5" | seu@meta.data$orig.ident == "mouseSampleTag6")] <- "aCD4"
seu@meta.data$sample.origin[which(seu@meta.data$orig.ident == "mouseSampleTag1")] <- "Control"
seu@meta.data$sample.origin[which(seu@meta.data$orig.ident == "mouseSampleTag2")] <- "aPDL1"


In [57]:
###Pseudotime analysis for subset of clusters
sample.basename <- "Pooled"
dir.name <- "results/preprocess/Pseudotime"
start <- "1" #starting clusters
subcluster <- c(0,1,2,3,4)
red.use <- "umap"

############################# Processing layer ######################################
dir.create(dir.name, recursive = TRUE)

#Call seurat object and condition name
seu.sub <- subset(seu, idents = subcluster)

#Define PCs for Pseudotime analysis (Call the result of Jackstraw)
tmp = as.data.frame(seu@reductions$pca@jackstraw@overall.p.values)
tmp1 = tmp[tmp$Score>0.05,1]
dims= c(1:(min(tmp1)-1))

#Perform pseudotime analysis
seu <- Pseudotime.Min(seu, seu.sub, sample.name, dims, start, dir.name)

#save seurat object with pseudotime
file.name <- str_c(dir.seurat, sample.name, sep = "/") %>% str_c("_clust_pseudotime.rda", sep='')
save(seu, file=file.name)


class: SlingshotDataSet 

 Samples Dimensions
   28098         26

lineages: 2 
Lineage1: 1  4  3  0  
Lineage2: 1  4  2  

curves: 2 
Curve1: Length: 34.883	Samples: 22429.03
Curve2: Length: 28.966	Samples: 15775.05


### Add Gene Scores for Tex term, Tex prog, Cytotoxic

In [44]:
# Introduce Gene Scores using AddModuleScore function

#Put signature gene set into "Signatures" directory
dir.input <- "Signatures"
sample.name <- "Pooled"

########################## Processing layer #############################
#load Signature data
files  <- list.files(dir.input, pattern="Sig.")

for(i in files){
      i <- str_c(dir.input, i, sep = "/")
      gene_list <- read.table(i, header = TRUE)
      gene_list <- list(as.vector(gene_list$Genes))
      sig_name <- str_remove(i, dir.input)
      sig_name <- str_remove(sig_name, "/Sig.")
      sig_name <- str_remove(sig_name, ".txt")
      seu <- AddModuleScore(object = seu, 
                            ctrl=10,
                            features = gene_list,
                            name=sig_name)
    }
#Output Seurat object
file.name <- str_c(dir.seurat, sample.name, sep = "/") %>% str_c("_clust_pseudotime_Signature.rda", sep='')
save(seu, file=file.name)

Warning message:
“The following features are not present in the object: Cd244, not searching for symbol synonyms”


### Integration of scTCRseq results into SCT Seurat object

In [45]:
###Define function
#Create histogram was showing the total number of TCR reads detected in each cell (top left) 
#and the percentage of TCR reads that were the most common among them (bottom left).
#Scatter plot representing these two metrices of cells were also generated
Histogram <- function(d, file.name, sample.name, dir.output){

  d_count_total <- as.numeric(d$count.total)
  
  #Count histogram
  ppi <- 300
  image.file <- str_c(dir.output, file.name, sep = "/") %>% str_c(sample.name, "histogram.count.tiff", sep = '.') 
  tiff(image.file, width=1.2*ppi, height=0.8*ppi, res=ppi)
  p <- ggplot(NULL, aes(x=d_count_total)) +
    geom_histogram(binwidth=0.1) +
    theme_bw(base_size = 6) +
    labs(x = "Read counts of TCR") +
    theme(
      axis.title.y=element_blank(),
      axis.text.x = element_text(family="Arial"),
      axis.text.y = element_text(family="Arial"),
      axis.title=element_text(size=4)) +
    scale_x_continuous(trans=scales::log2_trans(),
                   breaks=scales::trans_breaks("log2",function(x) 2^x),
                   labels=scales::trans_format("log2",scales::math_format(2^.x)))
  print(p)
  dev.off()

  #Frequency histogram  
  d_freq <- as.numeric(d$freq)
  med_f <- 0.3
  ppi <- 300
  image.file <- str_c(dir.output, file.name, sep = "/") %>% str_c(sample.name, "histogram.freq.tiff", sep = '.')
  tiff(image.file, width=1.2*ppi, height=0.8*ppi, res=ppi)
  p <- ggplot(NULL, aes(x=d_freq)) +
    geom_histogram(binwidth=0.05) +
    theme_bw(base_size = 6) +
    labs(x = "Proportion of the largest TCR read") +
    theme(
      axis.title.y=element_blank(),
      axis.text.x = element_text(family="Arial"),
      axis.text.y = element_text(family="Arial"),
      axis.title=element_text(size=4)) 
  print(p)
  dev.off()

  df <- data.frame(d_count_total, d_freq)
  
  #Scatter plot    
  total <- nrow(subset(df, df$d_freq >= 0))
  lb <- paste(round(100*nrow( subset(df, df$d_freq < 0.6 & df$d_count_total < 2^5 )) / total, digits = 1), "%", sep="" ) 
  rb <- paste(round(100*nrow( subset(df, df$d_freq >= 0.6 & df$d_count_total < 2^5 )) / total, digits = 1), "%", sep="" ) 
  lt <- paste(round(100*nrow( subset(df, df$d_freq < 0.6 & df$d_count_total >= 2^5 )) / total, digits = 1), "%", sep="" ) 
  rt <- paste(round(100*nrow( subset(df, df$d_freq >= 0.6 & df$d_count_total >= 2^5 )) / total, digits = 1), "%", sep="" ) 
  
  ppi <- 300
  image.file <- str_c(dir.output, file.name, sep = "/") %>% str_c(sample.name, "Scatter.tiff", sep = '.')                     
  tiff(image.file, width=1.2*ppi, height=1.2*ppi, res=ppi)
  p <- ggplot(d, aes(x=d_freq, y=d_count_total)) +  
    stat_bin2d(bins=60) +
    scale_fill_gradient(low="lightblue", high="red") +
    theme_bw(base_size = 6) +
    geom_vline(aes(xintercept = 0.6), size=0.25, colour="black") +
    geom_hline(aes(yintercept = 2^5), size=0.25, colour="black") +
    labs(x = "Proportion of the largest TCR read", y = "Read counts of TCR") +
    theme(
      axis.text.x = element_text(family="Arial"),
      axis.text.y = element_text(family="Arial"),
      axis.title=element_text(size=4)) +
    guides(fill=FALSE) +
    scale_y_continuous(trans=scales::log2_trans(),
                   breaks=scales::trans_breaks("log2",function(x) 2^x),
                   labels=scales::trans_format("log2",scales::math_format(2^.x)))
    p <- p + #xlim(0,1) +
    annotate("text", x=-Inf, y=0, hjust=-0.1, vjust=-0.4, label=lb,
             family="Arial",colour="black",size=1.5) +
    annotate("text", x=Inf, y=0, hjust=1.1, vjust=-0.4, label=rb,
             family="Arial",colour="black",size=1.5) +
    annotate("text", x=-Inf, y=Inf, hjust=-0.1, vjust=1.3, label=lt,
             family="Arial",colour="black",size=1.5) +
    annotate("text", x=Inf, y=Inf, hjust=1.1, vjust=1.3, label=rt,
             family="Arial",colour="black",size=1.5) 
  print(p)
  dev.off()  
}

In [46]:
###Define function
#Create TCRa/TCRb combine table for cell barcodes
CombineTable <- function(dir.input, sample.name, dir.output, count_th, freq_th, clonotype, cores){
    combined.tables <- data.frame()
    #Unzip files
    files <- list.files(dir.input, sample.name)
    for(file in files){
        name.input <- str_c(dir.input, file, sep = "/")
        bunzip2(name.input, remove=FALSE)
        name.input <- str_remove(name.input, pattern = ".bz2")
        untar(name.input)
        file.remove(name.input)
    }
    
    ###Define function
    tableread_fast = function(i, header=TRUE, quote="", sep=","){
      tmp = fread(i, header=header, sep=sep, quote=quote, nThread=32)
      tmp = as.data.frame(tmp)
      return(tmp)
    }
    
    #Convert MiXCR output into VDJtools format
    Convert <- function(file.name, name.TRA){
        name.input <- str_c(name.TRA, file.name, sep = "/")
        data <- tableread_fast(name.input, header = TRUE, sep = '\t')

        #Extract TCR information from mixcr output
        count.total<-data$cloneCount / data$cloneFraction
        freq<-data$cloneFraction
        cdr3nt<-data$nSeqImputedCDR3
        cdr3aa<-data$aaSeqImputedCDR3
        v<-str_sub(data$bestVHit, end=-4)
        d<-str_sub(data$bestDHit, end=-4)
        j<-str_sub(data$bestJHit, end=-4)
        data3 <- rbind(count.total,freq,cdr3nt,cdr3aa,v,d,j)
        data3 <- as.data.frame(t(data3))
        names(data3) <- c("count.total","freq","cdr3nt","cdr3aa","v","d","j")

        #Extract largest clone in cell barcode and append cell barcode information
        name_out <- str_split(file.name, "_")
        CB <- name_out[[1]][[4]]
        CB_out <- str_split(CB, "\\.")
        CB <- CB_out[[1]][[1]]
        d_out <- data3[1,]
        d_out$CB <- CB

        return(d_out)
    }

    #Convert MiXCR output into VDJtools format
    name.TRA <- str_c(sample.name, "TRAC1_mixcr", sep = "_")
    files  <- list.files(name.TRA, pattern=".txt")
    cl <- makeCluster(cores)
    registerDoParallel(cl)
    TRA.table <- invisible(foreach(file.name = files,
            .combine = rbind, .packages=c("ggplot2", "extrafont", "stringr", "dplyr", "data.table")) %dopar% {Convert(file.name, name.TRA)})
    stopCluster(cl)
    unlink(name.TRA, recursive=TRUE)

    name.TRB <- str_c(sample.name, "TRBC1_mixcr", sep = "_")
    files  <- list.files(name.TRB, pattern=".txt")
    cl <- makeCluster(cores)
    registerDoParallel(cl)
    TRB.table <- invisible(foreach(file.name = files,
            .combine = rbind, .packages=c("ggplot2", "extrafont", "stringr", "dplyr", "data.table")) %dopar% {Convert(file.name, name.TRB)})
    stopCluster(cl)
    unlink(name.TRB, recursive=TRUE)

    #Output histogram and scatter plot for summarizing scTCR status
    Histogram(TRA.table, "TRA", sample.name, dir.output)
    Histogram(TRB.table, "TRB", sample.name, dir.output)

    #thresholding
    TRA.table$count.total <- as.numeric(TRA.table$count.total)
    TRA.table$freq <- as.numeric(TRA.table$freq)
    TRB.table$count.total <- as.numeric(TRB.table$count.total)
    TRB.table$freq <- as.numeric(TRB.table$freq)
    TCRa_th <- subset(TRA.table, count.total >= 2^(count_th) & freq >= freq_th)
    TCRb_th <- subset(TRB.table, count.total >= 2^(count_th) & freq >= freq_th)

    #Paring TCRa and TCRb by cell barcode
    names(TCRa_th) <- c("count.total.A","freq.A","cdr3nt.A","cdr3aa.A","v.A","d.A","j.A", "CB")
    names(TCRb_th) <- c("count.total.B","freq.B","cdr3nt.B","cdr3aa.B","v.B","d.B","j.B", "CB")
    combined <- merge(TCRa_th, TCRb_th, all=T, by ="CB")
    combined$CB <- str_c(sample.name, combined$CB, sep = "_")
    #Exclude cells in which neither TCRa nor TCRb sequence were detected
    combined <- subset(combined, combined$cdr3nt.A != "UD" | combined$cdr3nt.B != "UD")

    #Generate clone id for further analysis
    #Definition of clones can be changed between "ABnt", "ABaa", "Bnt", and "Baa"
    if(clonotype=="nt"){
        combined$clone.id.TCRa <- str_c(combined$cdr3nt.A, combined$v.A, combined$j.A, sep="_")
        combined$clone.id.TCRb <- str_c(combined$cdr3nt.B, combined$v.B, combined$j.B, sep="_")
    }
    if(clonotype=="aa"){
        combined$clone.id.TCRa <- str_c(combined$cdr3aa.A, combined$v.A, combined$j.A, sep="_")
        combined$clone.id.TCRb <- str_c(combined$cdr3aa.B, combined$v.B, combined$j.B, sep="_")
    }
    combined$clone.id.TCRab <- str_c(combined$clone.id.TCRa, combined$clone.id.TCRb, sep = "_")
    name.output <- str_c(dir.output, sample.name, sep = "/") %>% str_c(clonotype, "table", "count_th", count_th, "freq_th", freq_th, clonotype, "csv", sep = ".")
    write.csv(combined, name.output, row.names = FALSE)
    
    combined.tables <- rbind(combined.tables, combined)
    
    return(combined.tables)
}

In [47]:
#Extract TCR sequence for each cel barcode
#iteration process for dataset (CD4/CD8 Seurat object) and clonotype (Ant/Bnt)
set.seed(seed = 42)
sample.list <- c("Ex124TCRBD", "ICIrevTCR1")
dir.input <- "data/scTCR_rawdata"
cores <- 12
dir.output <- "results/preprocess/scTCR_processing"
#Threshold for valid cell barcode with single TCR sequence
count_th <- 5 #read count threshold for all TCR sequences per cell barcode
freq_th <- 0.6 #proportion threshold for largest TCR
clonotype <- "nt" #Definition of clones. nt: nucleotide sequence / aa: amino acid sequnece 

################################ Processing layer #################################################
dir.create(dir.output)

combined.tables <- data.frame()

#Create TCRa/TCRb paring table
for(sample.name in sample.list){
    #Unzip files
    ##Create TCRa/TCRb combine table for cell barcodes
    combined.table <- CombineTable(dir.input, sample.name, dir.output, count_th, freq_th, clonotype, cores)
    combined.tables <- rbind(combined.tables, combined.table)
}

Warning message:
“Removed 170 rows containing non-finite values (stat_bin).”
Warning message:
“Removed 170 rows containing non-finite values (stat_bin).”
Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”
Warning message:
“Transformation introduced infinite values in continuous y-axis”
Warning message:
“Transformation introduced infinite values in continuous y-axis”
Warning message:
“Removed 170 rows containing non-finite values (stat_bin2d).”
Warning message:
“Removed 156 rows containing non-finite values (stat_bin).”
Warning message:
“Removed 156 rows containing non-finite values (stat_bin).”
Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”
Warning message:
“Transformation introduced infinite values in continuous y-axis”
Warning message:
“Transformation introduced infinite values in continuous y-axis”
Warning message:
“Removed 156 rows containing non-finite values (stat_b

In [48]:
###Integrate scTCRdata into the Seurat object
sample.name <- "Pooled"

################################ Processing layer #################################################
#Change batch name in combine.table
combined.tables$CB <- str_replace(combined.tables$CB, "Ex124TCRBD", "Aoki")
combined.tables$CB <- str_replace(combined.tables$CB, "TCR", "Target")

#Extract meta.data and cell BC information
meta.data <- seu@meta.data
meta.data$names <- row.names(meta.data)
CB.info <- str_split(meta.data$names, pattern = "_", simplify = TRUE)
meta.data$CB <- str_c(CB.info[,2], CB.info[,3], sep = "_")

#Merge to Seurat object of SCT data
meta.data <- merge(combined.tables, meta.data, all.y = T, by ="CB")
clone.ids <- c("clone.id.TCRa", "clone.id.TCRb", "clone.id.TCRab")
for(clone.id in clone.ids){
    clone.id.list <- dplyr::select(meta.data, clone.id)
    row.names(clone.id.list)=as.character(meta.data$names)
    seu <- AddMetaData(object = seu, metadata = clone.id.list, col.name = clone.id)
}
    
#Output Seurat object
file.name <- str_c(dir.seurat, sample.name, sep = "/") %>% str_c("_clust_pseudotime_Signature_scTCR.rda", sep='')
save(seu, file=file.name)

### SCT analysis after integrating scTCRseq results

In [49]:
###Define function

##Make clone table by assembling clones from scTCR results 
CloneSummary <- function(seu, dir.name, sample.name, clonotype){
    #load metadata
    meta.data <- as.data.frame(seu@meta.data)
    meta.data$CellBC <- row.names(meta.data)
    
    #extract cells only in the assigned sample
    meta.data <- dplyr::filter(meta.data, str_detect(sample.origin, sample.name))
    
    #Define clone by TCRa / TCRb/ TCRa&b
    if(clonotype == "TCRa"){
       meta.data$clone.id <- meta.data$clone.id.TCRa 
    }
    if(clonotype == "TCRb"){
       meta.data$clone.id <- meta.data$clone.id.TCRb 
    }
    if(clonotype == "TCRab"){
       meta.data$clone.id <- meta.data$clone.id.TCRab 
    } 

    #Summarize cluster distribution of each clone
    tmp_out = table(meta.data$seurat_clusters, meta.data$clone.id)
    tmp_out2 <- as.data.frame(tmp_out, row.names = NULL,
                  responseName = "Freq", stringsAsFactors = TRUE,
                  sep = "", base = list(LETTERS))
    names(tmp_out2) <- c("Clust", "names", "Freq")
    tmp_out2 <- dcast(tmp_out2, names ~ Clust)

    #Summarize the count and frequency of clones 
    tmp = meta.data %>% group_by(`clone.id`) %>%
      dplyr::summarise(count = n()) %>%
      dplyr::arrange(desc(count))
    tmp = as.data.frame(tmp)
    tmp <- tmp[!is.na(tmp$clone.id), ] #NAである細胞を除く
    tmp[,3]=tmp[,2]/sum(tmp[,2])
    colnames(tmp)=c("ntSeq_TRA_TRB_freq", "CloneCount", "CloneFreq")
    tmp <- tmp[order(tmp$ntSeq_TRA_TRB_freq),]
    temp_count <- as.vector(tmp$CloneCount)
    temp_freq <- as.vector(tmp$CloneFreq)
    #Combine to the cluster distribution
    tmp_out3 <- cbind(temp_count, temp_freq, tmp_out2)

    #Assign ranks to clones
    tmp_out3 <- tmp_out3[order(tmp_out3$temp_freq, decreasing=T),]
    rank <-  1:nrow(tmp_out3)
    tmp_out3 <- cbind(rank, tmp_out3)
    tmp_out3 <- data.frame(tmp_out3)
    tmp_out3$rank <- paste("Top",tmp_out3$rank, sep="")
    
    #Output
    name.out <- str_c(dir.name, sample.name, sep = "/") %>% str_c(clonotype, "clone_within_cluster.txt", sep = ".")
    write.table(tmp_out3, name.out, row.names=F, col.names=T, sep="\t", quote=F) 
}

In [50]:
# Main module

###Input layer 
sample.name <- "Pooled"
dir.name <- "results/preprocess/scTCR_analysis"
treats <- c("Control", "aPDL1", "aCD4")
clonotype <- "nt" #Definition of clones. nt: nucleotide sequence / aa: amino acid sequnece 

########################## Processing layer #############################
dir.create(dir.name, recursive = TRUE)
    
#Make clone table by assembling clones from scTCR results 
for(treat in treats){
    CloneSummary(seu, dir.name, treat, "TCRb") 
}

Using Freq as value column: use value.var to override.

Using Freq as value column: use value.var to override.

Using Freq as value column: use value.var to override.



In [51]:
### Detect dLN-Tumor OL clones

###Input layer
#Specify query files: frequency and cluster distribution table of scTCR clones
dir.name <- "results/preprocess/scTCR_analysis"
treats <- c("aCD4", "aPDL1", "Control")
#Specify the repertoire data of dLN
dir.bulk <- "data/BulkTCR_rawdata"

cores <- 4

########################## Processing layer #############################
dir.create(dir.name)
    
for(treat in treats){
    #load scTCR clone table file
    name.query <- str_c(dir.name, treat, sep = "/") %>% str_c("TCRb.clone_within_cluster.txt", sep = ".") 
    clone.q <- read.table(name.query, header = TRUE)
    clone.q$TCRb <- clone.q$names

    #load dLN reprtoire files
    name.bulk <- str_c(dir.bulk, "CD8_dLN_SCT_", sep = "/") %>% str_c(treat, ".txt", sep = "")
    d <- tableread_fast(name.bulk, header = TRUE, sep="\t", quote = "\"")
    #Reconstruct the information of clones for searching overlap
    d$TCRb <- str_c(d$cdr3nt, d$v, d$j, sep = "_")
    d <- dplyr::select(d, c("TCRb", "freq"))
    names(d) <- c("TCRb", "freq_dLN")

    #Search clones overlapped with Tumor scTCR repertoire
    d_output <- merge(clone.q, d, by = "TCRb", all.x = T)
    d_output$treat <- treat

    ###Define Oligoclonal / Polyclonal fraction of dLN-tumor OL
    d_output$OL <- "nonOL"
    #Extract OL clones
    data_LT <- dplyr::filter(d_output, !is.na(freq_dLN))
    data_nonOL <- dplyr::filter(d_output, is.na(freq_dLN))
    #Determine Oligoclonal / Polyclonal fraction
    data_LT <- data_LT[order(data_LT$temp_freq, decreasing = T),]
    count.poly <- nrow(data_LT)-10
    data_LT$OL <- c(rep("Oligo", times = 10), rep("Poly", times = count.poly))
    d_output <- rbind(data_LT, data_nonOL)
        
    ### Integrate dLN-tumor OL status (oligoclona OL / polyclonal OL / nonOL) and gene scores
    ###Summarize Gene score for each TCR clone
    meta.data <- as.data.frame(seu@meta.data)
    meta.data.calc <- dplyr::filter(meta.data, sample.origin == treat) %>% dplyr::select(c("clone.id.TCRb", "TexKLR1", "Tumor.Prog1", "Tumor.Term1"))
    out_score <- aggregate(x=meta.data.calc[c("TexKLR1", "Tumor.Prog1", "Tumor.Term1")],
                         by=list(meta.data.calc$clone.id), FUN=mean)
    #Integration
    out <- merge(d_output, out_score, by.x = "names", by.y = "Group.1", all.x = T)

    name.out <- str_c(dir.name, treat, sep = "/") %>% str_c("scTCR.dLNOL.csv", sep = ".")
    write.csv(out, name.out, row.names = FALSE)
}

Warning message in dir.create(dir.name):
“'results/preprocess/scTCR_analysis' already exists”
